<a href="https://colab.research.google.com/github/Maryam-Mostafa/Natural-scene-classification-using-different-CNN-architecture/blob/master/NaturalSceneClassification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Liberaries**

In [ ]:
!pip install tflearn
from google.colab import drive
from google.colab import files
import os
import numpy as np
import pandas as pd
import cv2
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
import tflearn
from keras.utils import np_utils
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from sklearn.model_selection import train_test_split
drive.mount("/content/drive")

### **Preprocessing**

In [ ]:
def Read_Data(path_,label,isTrain):
    sub_data = []
    for img in tqdm(os.listdir(path_)):
        path = os.path.join(path_, img)
        img_data = cv2.imread(path, 0)
       img_data = cv2.resize(img_data, (150, 150)) 
        if isTrain:
          sub_data.append([np.array(img_data), label])
        else:
          sub_data.append([np.array(img_data), img])
    return sub_data

def Creat_all_train_data():
    training_data,sub1,sub2,sub3,sub4,sub5,sub6 = [],[],[],[],[],[],[]
          
    sub1 = Read_Data("/content/drive/MyDrive/NNP/Scenetrainingset/buildings/",0,True)
    sub2 = Read_Data("/content/drive/MyDrive/NNP/Scenetrainingset/forest",1,True)
    sub3 = Read_Data("/content/drive/MyDrive/NNP/Scenetrainingset/glacier",2,True)
    sub4 = Read_Data("/content/drive/MyDrive/NNP/Scenetrainingset/mountain",3,True)
    sub5 = Read_Data("/content/drive/MyDrive/NNP/Scenetrainingset/sea",4,True)
    sub6 = Read_Data("/content/drive/MyDrive/NNP/Scenetrainingset/street",5,True)
    training_data = sub1 + sub2 + sub3 + sub4 + sub5 + sub6
    shuffle(training_data)
    np.save('/content/drive/MyDrive/NNP/train_dataset.npy', training_data)

    return training_data

if (os.path.exists('/content/drive/MyDrive/NNP/train_dataset.npy')): 
    train =np.load('/content/drive/MyDrive/NNP/train_dataset.npy',allow_pickle=True)

else: 
    train = Creat_all_train_data()

if (os.path.exists('/content/drive/MyDrive/NNP/test_dataset.npy')): 
    test =np.load('/content/drive/MyDrive/NNP/test_dataset.npy',allow_pickle=True)

else: 
      test = Read_Data("/content/drive/MyDrive/NNP/Scenestestingtest",None,False)
      np.save('/content/drive/MyDrive/NNP/test_dataset.npy', test)

X_train = np.array([i[0] for i in train]).reshape(-1, 150, 150, 1)
Y_train = np.array([i[1] for i in train])
Y_train = np_utils.to_categorical(Y_train, num_classes=6)


X_test = np.array([i[0] for i in test]).reshape(-1, 150, 150, 1)
img_name = np.array([i[1] for i in test])



### **Model architecture**

In [ ]:
ops.reset_default_graph()
conv_input = input_data(shape=[None, 150, 150, 1], name='input')
conv1 = conv_2d(conv_input, 32, 5, activation='relu')
pool1 = max_pool_2d(conv1, 5)

conv2 = conv_2d(pool1, 64, 5, activation='relu')
pool2 = max_pool_2d(conv2, 5)

conv3 = conv_2d(pool2, 128, 5, activation='relu')
pool3 = max_pool_2d(conv3, 5)

conv4 = conv_2d(pool3, 64, 5, activation='relu')
pool4 = max_pool_2d(conv4, 5)

conv5 = conv_2d(pool4, 32, 5, activation='relu')
pool5 = max_pool_2d(conv5, 5)

conv6 = conv_2d(pool5, 64, 5, activation='relu')
pool6 = max_pool_2d(conv6, 5)

conv7 = conv_2d(pool6, 32, 5, activation='relu')
pool7 = max_pool_2d(conv7, 5)

conv8 = conv_2d(pool7, 32, 5, activation='relu')
pool8 = max_pool_2d(conv8, 5)

fully_layer = fully_connected(pool8, 1500, activation='relu')
fully_layer = dropout(fully_layer, 0.5)

cnn_layers = fully_connected(fully_layer, 6, activation='softmax')

### **Training the model and geting prediction**

In [ ]:
cnn_layers = regression(cnn_layers, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy', name='targets')
model = tflearn.DNN(cnn_layers, tensorboard_dir='log', tensorboard_verbose=3)
model.fit( X_train, Y_train, n_epoch=40,snapshot_step=100, show_metric=True, run_id='cnn')

prediction = model.predict(X_test)
prediction = np.argmax(prediction, axis=1)

### **saving the result in csv file**

In [ ]:
rows = {'Image': [],'Label': []}
csvFile = pd.DataFrame(rows, columns = ['Image', 'Label'])

csvFile["Image"] = img_name
csvFile["Label"] = np.asarray(prediction)

csvFile.to_csv('submit2.csv')
files.download('submit2.csv')